In [1]:
import pandas as pd
import torch
import sklearn as sk
from sklearn import preprocessing
import time
import numpy as np

# Load the database, shouldn't take that long. This is all data!

t1 = time.time()
print('Loading database ...')
df = pd.read_hdf('database/all_data_comp.h5','table')
print('Time to load database:', time.time()-t1)


Loading database ...
Time to load database: 4.280972242355347


In [2]:

#Creating the Tensors:
#
#for name_ in var_names:
#    globals()[name_] = 3

fo_1_3_total = df['FO_day_engine_1_3'].dropna()
fo_2_4_total = df['FO_day_engine_2_4'].dropna()

# this gives the total in tonnes for each day.

In [3]:
##
# The cost function
# The dataset is not complete overlapping in time with data from both the mass-flow meters and the
# the rest of the data. So we have to manually filter out the time interval which we are interested in.

date_begin = '2014-02-01'
date_end = '2014-12-16'

# Dict of var names we want to use.


var_names = {'ae1_frp':'AE1 FUEL RACK POSIT:1742:mm:Average:900',
             'ae2_frp':'AE2 FUEL RACK POSIT:2742:mm:Average:900',
             'ae3_frp':'AE3 FUEL RACK POSIT:3742:mm:Average:900',
             'ae4_frp':'AE4 FUEL RACK POSIT:4742:mm:Average:900',
             
             'ae1_cac_P':'AE1 CA OTL CAC PRESS:1649:Bar:Average:900',
             'ae2_cac_P':'AE2 CA OTL CAC PRESS:2649:Bar:Average:900',
             'ae3_cac_P':'AE3 CA OTL CAC PRESS:3649:Bar:Average:900',
             'ae4_cac_P':'AE4 CA OTL CAC PRESS:4649:Bar:Average:900',
             
             'ae1_cac_ca':'AE1 EXH CA OUTET 1:1543:  C:Average:900',
             'ae2_cac_ca':'AE2 EXH CA OUTET 1:2543:  C:Average:900',
             'ae3_cac_ca':'AE3 EXH CA OUTET 1:3543:  C:Average:900',
             'ae4_cac_ca':'AE4 EXH CA OUTET 1:4543:  C:Average:900',
             
             'ae1_exh':'AE1 EXH MEAN VALUE:1591:  C:Average:900',
             'ae2_exh':'AE2 EXH MEAN VALUE:2591:  C:Average:900',
             'ae3_exh':'AE3 EXH MEAN VALUE:3591:  C:Average:900',
             'ae4_exh':'AE4 EXH MEAN VALUE:4591:  C:Average:900',
             
             'ae1_fo_P':'AE1 FO INLET PRESS:1603:Bar:Average:900',
             'ae2_fo_P':'AE2 FO INLET PRESS:2603:Bar:Average:900',
             'ae3_fo_P':'AE3 FO INLET PRESS:3603:Bar:Average:900',
             'ae4_fo_P':'AE4 FO INLET PRESS:4603:Bar:Average:900',
             
             'ae1_rpm':'AE1 ENG SPEED:1745:RPM:Average:900',
             'ae2_rpm':'AE2 ENG SPEED:2745:RPM:Average:900',
             'ae3_rpm':'AE3 ENG SPEED:3745:RPM:Average:900',
             'ae4_rpm':'AE4 ENG SPEED:4745:RPM:Average:900',
             
             'me1_frp':'ME1 FUEL RACK POSIT:10005:%:Average:900',
             'me2_frp':'ME2 FUEL RACK POSIT:20005:%:Average:900',
             'me3_frp':'ME3 FUEL RACK POSIT:30005:%:Average:900',
             'me4_frp':'ME4 FUEL RACK POSIT:40005:%:Average:900',
             
             'me1_ca_T':'ME1 CA TEMP COOL OUT:1343:C:Average:900',
             'me2_ca_T':'ME2 CA TEMP COOL OUT:2343:C:Average:900',
             'me3_ca_T':'ME3 CA TEMP COOL OUT:3343:C:Average:900',
             'me4_ca_T':'ME4 CA TEMP COOL OUT:4343:C:Average:900',
             
             'me1_cac_T':'ME1 CHARGE AIR TEMP:1347:C:Average:900',
             'me2_cac_T':'ME2 CHARGE AIR TEMP:2347:C:Average:900',
             'me3_cac_T':'ME3 CHARGE AIR TEMP:3347:C:Average:900',
             'me4_cac_T':'ME4 CHARGE AIR TEMP:4347:C:Average:900',
             
             'me1_exh_T':'ME1 EXH GAS MEAN:1125:C:Average:900',
             'me2_exh_T':'ME2 EXH GAS MEAN:2125:C:Average:900',
             'me3_exh_T':'ME3 EXH GAS MEAN:3125:C:Average:900',
             'me4_exh_T':'ME4 EXH GAS MEAN:4125:C:Average:900',
             
             'me1_rpm':'ME1 ENGINE SPEED:1364:rpm:Average:900',
             'me2_rpm':'ME2 ENGINE SPEED:2364:rpm:Average:900',
             'me3_rpm':'ME3 ENGINE SPEED:3364:rpm:Average:900',
             'me4_rpm':'ME4 ENGINE SPEED:4364:rpm:Average:900',
             
             'fo_booster_13':'FO BOOST 1 CONSUMPT:6165:m3/h:Average:900',
             'fo_booster_24':'FO BOOST 2 CONSUMPT:6166:m3/h:Average:900'}

for names in var_names:
    if var_names[names] in list(df):
        print(var_names[names])
    else:
        print('*** VAR MISSING *** ', var_names[names], ' *** VAR MISSING ***')

        
eng_13 = [var_names['ae1_frp'],
          var_names['ae3_frp'],
          var_names['ae1_cac_P'],
          var_names['ae3_cac_P'],
          var_names['ae1_cac_ca'],
          var_names['ae3_cac_ca'],
          var_names['ae1_exh'],
          var_names['ae3_exh'],
          var_names['ae1_fo_P'],
          var_names['ae3_fo_P'],
          var_names['ae1_rpm'],
          var_names['ae3_rpm'],
          var_names['me1_frp'],
          var_names['me3_frp'],
          var_names['me1_ca_T'],
          var_names['me3_ca_T'],
          var_names['me1_cac_T'],
          var_names['me3_cac_T'],
          var_names['me1_exh_T'],
          var_names['me3_exh_T'],
          var_names['me1_rpm'],
          var_names['me3_rpm']]
          
eng_24 = [var_names['ae2_frp'],
          var_names['ae4_frp'],
          var_names['ae2_cac_P'],
          var_names['ae4_cac_P'],
          var_names['ae2_cac_ca'],
          var_names['ae4_cac_ca'],
          var_names['ae2_exh'],
          var_names['ae4_exh'],
          var_names['ae2_fo_P'],
          var_names['ae4_fo_P'],
          var_names['ae2_rpm'],
          var_names['ae4_rpm'],
          var_names['me2_frp'],
          var_names['me4_frp'],
          var_names['me2_ca_T'],
          var_names['me4_ca_T'],
          var_names['me2_cac_T'],
          var_names['me4_cac_T'],
          var_names['me2_exh_T'],
          var_names['me4_exh_T'],
          var_names['me2_rpm'],
          var_names['me4_rpm']]
        

# Create np arrays. X vals are engine inputs, y are fo flows    

X_13 = np.array(df[[
            var_names['ae1_frp'],
            var_names['ae3_frp'],
            var_names['me1_frp'],
            var_names['me3_frp'],
            var_names['ae1_rpm'],
            var_names['ae3_rpm'],
            var_names['me1_rpm'],
            var_names['me3_rpm'],       
            ]][date_begin:date_end])
X_24 = np.array(df[[
            var_names['ae2_frp'],
            var_names['ae4_frp'],
            var_names['me2_frp'],
            var_names['me4_frp'],
            var_names['ae2_rpm'],
            var_names['ae4_rpm'],
            var_names['me2_rpm'],
            var_names['me4_rpm'],
            ]][date_begin:date_end])

#X_13 = np.array(df[eng_13][date_begin:date_end])
#X_24 = np.array(df[eng_24][date_begin:date_end])

y_13 = np.array(df[var_names['fo_booster_13']][date_begin:date_end]).reshape(-1,1)
y_24 = np.array(df[var_names['fo_booster_24']][date_begin:date_end]).reshape(-1,1)


AE1 FUEL RACK POSIT:1742:mm:Average:900
AE2 FUEL RACK POSIT:2742:mm:Average:900
AE3 FUEL RACK POSIT:3742:mm:Average:900
AE4 FUEL RACK POSIT:4742:mm:Average:900
AE1 CA OTL CAC PRESS:1649:Bar:Average:900
AE2 CA OTL CAC PRESS:2649:Bar:Average:900
AE3 CA OTL CAC PRESS:3649:Bar:Average:900
AE4 CA OTL CAC PRESS:4649:Bar:Average:900
AE1 EXH CA OUTET 1:1543:  C:Average:900
AE2 EXH CA OUTET 1:2543:  C:Average:900
AE3 EXH CA OUTET 1:3543:  C:Average:900
AE4 EXH CA OUTET 1:4543:  C:Average:900
AE1 EXH MEAN VALUE:1591:  C:Average:900
AE2 EXH MEAN VALUE:2591:  C:Average:900
AE3 EXH MEAN VALUE:3591:  C:Average:900
AE4 EXH MEAN VALUE:4591:  C:Average:900
AE1 FO INLET PRESS:1603:Bar:Average:900
AE2 FO INLET PRESS:2603:Bar:Average:900
AE3 FO INLET PRESS:3603:Bar:Average:900
AE4 FO INLET PRESS:4603:Bar:Average:900
AE1 ENG SPEED:1745:RPM:Average:900
AE2 ENG SPEED:2745:RPM:Average:900
AE3 ENG SPEED:3745:RPM:Average:900
AE4 ENG SPEED:4745:RPM:Average:900
ME1 FUEL RACK POSIT:10005:%:Average:900
ME2 FUEL RAC

In [23]:
# https://github.com/MorvanZhou/PyTorch-Tutorial/blob/master/tutorial-contents/301_regression.py

from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt

X_13 = torch.Tensor(np.array(df[[
            var_names['ae1_frp'],
            var_names['ae3_frp'],
            var_names['me1_frp'],
            var_names['me3_frp'],
            var_names['ae1_rpm'],
            var_names['ae3_rpm'],
            var_names['me1_rpm'],
            var_names['me3_rpm'],       
            ]][date_begin:date_end]))
X_24 = torch.Tensor(np.array(df[[
            var_names['ae2_frp'],
            var_names['ae4_frp'],
            var_names['me2_frp'],
            var_names['me4_frp'],
            var_names['ae2_rpm'],
            var_names['ae4_rpm'],
            var_names['me2_rpm'],
            var_names['me4_rpm'],
            ]][date_begin:date_end]))

y_13 = torch.Tensor(np.array(df[var_names['fo_booster_13']][date_begin:date_end]).reshape(-1,1))
y_24 = torch.Tensor(np.array(df[var_names['fo_booster_24']][date_begin:date_end]).reshape(-1,1))

#X_13 = torch.Tensor(X_13)
#y_13 = torch.Tensor(y_13)

X_13, y_13 = Variable(X_13), Variable(y_13)

class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

net = Net(n_feature=8, n_hidden=10, n_output=1)     # define the network
print(net)  # net architecture

Net (
  (hidden): Linear (8 -> 10)
  (predict): Linear (10 -> 1)
)


In [27]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

plt.ion()   # something about plotting

for t in range(100):
    prediction = net(X_13)     # input x and predict based on x

    loss = loss_func(prediction, y_13)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients

    if t % 5 == 0:
        print(loss.data[0])
        # plot and show learning process
        #plt.cla()
        #plt.scatter(X_13.data.numpy(), y_13.data.numpy())
        #plt.plot(X_13.data.numpy(), prediction.data.numpy(), 'r-', lw=5)
        #plt.text(0.5, 0, 'Loss=%.4f' % loss.data[0], fontdict={'size': 20, 'color':  'red'})
        #plt.pause(0.1)

#plt.ioff()
#plt.show()

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [29]:
loss.data


nan
[torch.FloatTensor of size 1]

In [7]:
y_13

Variable containing:
 0.4303
 0.4243
 0.4195
   ⋮    
 0.7703
 0.9352
 0.9189
[torch.FloatTensor of size 30624x1]

In [154]:
# Linear modeling with scikit

from sklearn import cross_validation
from sklearn.linear_model import LinearRegression


X_train_13, X_test_13, y_train_13, y_test_13 = cross_validation.train_test_split(X_13, y_13, test_size=0.2)
clf_13 = LinearRegression(n_jobs=-1)
clf_13.fit(X_train_13, y_train_13)
print('Model eng 1/3 :', clf_13.score(X_test_13,y_test_13))


X_train_24, X_test_24, y_train_24, y_test_24 = cross_validation.train_test_split(X_24, y_24, test_size=0.2)
clf_24 = LinearRegression(n_jobs=-1)
clf_24.fit(X_train_24, y_train_24)
print('Model eng 2/4 :', clf_24.score(X_test_24,y_test_24))



Model eng 1/3 : 0.963292871044
Model eng 2/4 : 0.978890310166


In [158]:
MSE = np.sum(np.square(clf_13.predict(X_13) - y_13))/len(y_13)
print('Model eng 1/3 with scikit-learn linear regression MSE:', MSE)

Model eng 1/3 with scikit-learn linear regression MSE: 0.0054041466284


In [160]:
0.0054041466284/0.00045995597151

11.749269415197727

In [131]:
# That was with only two predictors. Let's scale it up a bit...


X = np.array(df[date_begin:date_end].drop([var_names['fo_booster_13'],var_names['fo_booster_24']],1))

y_13 = np.array(df[var_names['fo_booster_13']][date_begin:date_end]).reshape(-1,1)
y_24 = np.array(df[var_names['fo_booster_24']][date_begin:date_end]).reshape(-1,1)


X_train, X_test, y_train_13, y_test_13 = cross_validation.train_test_split(X, y_13, test_size=0.2)
clf_13 = LinearRegression(n_jobs=-1)
clf_13.fit(X_train_13, y_train_13)
print('Model eng 1/3 :', clf_13.score(X_test_13,y_test_13))


X_train, X_test, y_train_24, y_test_24 = cross_validation.train_test_split(X, y_24, test_size=0.2)
clf_24 = LinearRegression(n_jobs=-1)
clf_24.fit(X_train_24, y_train_24)
print('Model eng 2/4 :', clf_24.score(X_test_24,y_test_24))

# This did not really scale well! Building a linear model with all datapoints does not even remotly work...



Model eng 1/3 : -0.000162105420374
Model eng 2/4 : 9.84541662015e-05


In [132]:
clf_24.coef_

array([[ -2.32256655e-06,   3.10425224e-04,  -5.03540170e-04,
          3.92613808e-05,   5.45276208e-06,  -4.97676582e-06,
          5.27064403e-05,  -1.07291175e-05]])

In [130]:
y_13.shape

(30624, 1)

In [65]:
x_in = np.array([20,20,30,50])

In [66]:
x_in.shape

(4,)

In [74]:
print(clf.predict(X_train[2].reshape(1,-1)))
print(y_train[2])

[[ 0.71141368]]
[ 0.79085722]


array([[ 0.006779  , -0.0025682 ,  0.01336264,  0.01303422]])

In [10]:



def day_predict(eng_side,day):
    np.sum()



Time
2014-02-01    12.15
2014-02-02    13.80
2014-02-03    12.30
2014-02-04    11.69
2014-02-05    16.00
2014-02-06    13.67
2014-02-07    10.95
2014-02-08    15.85
2014-02-09    14.02
2014-02-10    11.67
2014-02-11    14.61
2014-02-12    14.77
2014-02-13    12.81
2014-02-14    13.14
2014-02-15    10.69
2014-02-16    12.29
2014-02-17    12.31
2014-02-18    14.06
2014-02-19    10.16
2014-02-20    11.11
2014-02-21    13.84
2014-02-22    13.99
2014-02-23     9.11
2014-02-24    12.90
2014-02-25    13.07
2014-02-26    14.32
2014-02-27    10.63
2014-02-28     9.75
2014-03-01     9.70
2014-03-02    12.05
              ...  
2014-11-17    14.21
2014-11-18    13.35
2014-11-19    14.31
2014-11-20    11.08
2014-11-21     8.86
2014-11-22    10.12
2014-11-23    12.72
2014-11-24    12.26
2014-11-25    14.46
2014-11-26    12.34
2014-11-27    13.83
2014-11-28    11.09
2014-11-29    12.34
2014-11-30    13.17
2014-12-01    15.01
2014-12-02    17.46
2014-12-03     8.63
2014-12-04    12.78
2014-12-05    1